In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, default_data_collator
from datasets import load_dataset, load_metric, ClassLabel, Sequence
import transformers
import pandas as pd
from IPython.display import display, HTML
import random
import numpy as np
import collections
from tqdm.auto import tqdm

# Check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

transformers.__version__

Using cuda:0 device


'4.35.0'

In [3]:
squad_v2 = True
model_checkpoint = "distilbert-base-uncased"
batch_size = 32
max_length = 384
doc_stride = 128
n_best_size = 20
max_answer_length = 30

## Loading the dataset

In [4]:
datasets1={}

In [5]:
datasets1['train'] = load_dataset("squad_v2", split='train[:90%]' if squad_v2 else "squad")
datasets1['validation'] = load_dataset("squad_v2", split='train[90%:100%]' if squad_v2 else "squad")
datasets1['test'] = load_dataset("squad_v2", split='train' if squad_v2 else "squad")

In [6]:
datasets1

{'train': Dataset({
     features: ['id', 'title', 'context', 'question', 'answers'],
     num_rows: 117287
 }),
 'validation': Dataset({
     features: ['id', 'title', 'context', 'question', 'answers'],
     num_rows: 13032
 }),
 'test': Dataset({
     features: ['id', 'title', 'context', 'question', 'answers'],
     num_rows: 130319
 })}

In [7]:
from datasets import Dataset, DatasetDict
datasets= DatasetDict(datasets1)

In [8]:
datasets["train"][0]

{'id': '56be85543aeaaa14008c9063',
 'title': 'Beyoncé',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'question': 'When did Beyonce start becoming popular?',
 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}

In [9]:
datasets["validation"][0]

{'id': '5730b5b4069b531400832297',
 'title': 'Sumer',
 'context': 'Sumerian religion seems to have been founded upon two separate cosmogenic myths. The first saw creation as the result of a series of hieros gami or sacred marriages, involving the reconciliation of opposites, postulated as a coming together of male and female divine beings; the gods. This continued to influence the whole Mesopotamian mythos. Thus in the Enuma Elish the creation was seen as the union of fresh and salt water; as male Abzu, and female Tiamat. The product of that union, Lahm and Lahmu, "the muddy ones", were titles given to the gate keepers of the E-Abzu temple of Enki, in Eridu, the first Sumerian city. Describing the way that muddy islands emerge from the confluence of fresh and salty water at the mouth of the Euphrates, where the river deposited its load of silt, a second hieros gamos supposedly created Anshar and Kishar, the "sky-pivot" or axle, and the "earth pivot", parents in turn of Anu (the sky) an

In [10]:
# Show random elements in the dataset
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [11]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,57277c575951b619008f8b14,Tuberculosis,"It is the most widely used vaccine worldwide, with more than 90% of all children being vaccinated. The immunity it induces decreases after about ten years. As tuberculosis is uncommon in most of Canada, the United Kingdom, and the United States, BCG is administered only to those people at high risk. Part of the reasoning against the use of the vaccine is that it makes the tuberculin skin test falsely positive, reducing the test's use in screening. A number of new vaccines are currently in development.",What percentage of kids in the whole world get the BCG vaccine?,"{'text': ['90%'], 'answer_start': [61]}"
1,5a847d907cf838001a46a864,Endangered_Species_Act,"The ESA requires that critical habitat be designated at the time of or within one year of a species being placed on the endangered list. In practice, most designations occur several years after listing. Between 1978 and 1986 the FWS regularly designated critical habitat. In 1986 the Reagan Administration issued a regulation limiting the protective status of critical habitat. As a result, few critical habitats were designated between 1986 and the late 1990s. In the late 1990s and early 2000s, a series of court orders invalidated the Reagan regulations and forced the FWS and NMFS to designate several hundred critical habitats, especially in Hawaii, California and other western states. Midwest and Eastern states received less critical habitat, primarily on rivers and coastlines. As of December, 2006, the Reagan regulation has not yet been replaced though its use has been suspended. Nonetheless, the agencies have generally changed course and since about 2005 have tried to designate critical habitat at or near the time of listing.",What did the Reagan regulations invalidate?,"{'text': [], 'answer_start': []}"
2,571a5a3d10f8ca1400304feb,Ashkenazi_Jews,"It is estimated that in the 11th century Ashkenazi Jews composed only three percent of the world's Jewish population, while at their peak in 1931 they accounted for 92 percent of the world's Jews. Immediately prior to the Holocaust, the number of Jews in the world stood at approximately 16.7 million. Statistical figures vary for the contemporary demography of Ashkenazi Jews, oscillating between 10 million and 11.2 million. Sergio DellaPergola in a rough calculation of Sephardic and Mizrahi Jews, implies that Ashkenazi make up less than 74% of Jews worldwide. Other estimates place Ashkenazi Jews as making up about 75% of Jews worldwide.","By 1931, what percentage of the world’s Jews were Ashkenazi Jews?","{'text': ['92 percent'], 'answer_start': [165]}"
3,57281a552ca10214002d9dd7,Central_African_Republic,"Michel Djotodia took over as president and in May 2013 Central African Republic's Prime Minister Nicolas Tiangaye requested a UN peacekeeping force from the UN Security Council and on 31 May former President Bozizé was indicted for crimes against humanity and incitement of genocide. The security situation did not improve during June–August 2013 and there were reports of over 200,000 internally displaced persons (IDPs) as well as human rights abuses and renewed fighting between Séléka and Bozizé supporters.",What was Bozize indicted for?,"{'text': ['crimes against humanity'], 'answer_start': [232]}"
4,56f95a099b226e1400dd1389,Marshall_Islands,"Under German control, and even before then, Japanese traders and fishermen from time to time visited the Marshall Islands, although contact with the islanders was irregular. After the Meiji Restoration (1868), the Japanese government adopted a policy of turning the Japanese Empire into a great economic and military power in East Asia.",In what region did the Japanese attempt to build up their power after 1868?,"{'text': ['East Asia'], 'answer_start': [326]}"
5,5726cf69f1498d1400e8ebed,The_Sun_(United_Kingdom),"In August 2013, The Irish Sun ended the practice of featuring topless models on Page 3. The main newspaper was 

## Preprocessing the training data

In [12]:
# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [13]:
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [14]:
tokenizer("What is your name?", "My name is Minhah.")

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102, 2026, 2171, 2003, 19538, 4430, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [15]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

In [16]:
len(tokenizer(example["question"], example["context"])["input_ids"])

437

In [17]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

384

In [18]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [19]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 192]

In [20]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] beyonce got married in 2008 to whom? [SEP] on april 4, 2008, beyonce married jay z. she publicly revealed their marriage in a video montage at the listening party for her third studio album, i am... sasha fierce, in manhattan's sony club on october 22, 2008. i am... sasha fierce was released on november 18, 2008 in the united states. the album formally introduces beyonce's alter ego sasha fierce, conceived during the making of her 2003 single " crazy in love ", selling 482, 000 copies in its first week, debuting atop the billboard 200, and giving beyonce her third consecutive number - one album in the us. the album featured the number - one song " single ladies ( put a ring on it ) " and the top - five songs " if i were a boy " and " halo ". achieving the accomplishment of becoming her longest - running hot 100 single in her career, " halo "'s success in the us helped beyonce attain more top - ten singles on the list than any other woman during the 2000s. it also included the suc

In [21]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 7), (8, 11), (12, 19), (20, 22), (23, 27), (28, 30), (31, 35), (35, 36), (0, 0), (0, 2), (3, 8), (9, 10), (10, 11), (12, 16), (16, 17), (18, 25), (26, 33), (34, 37), (38, 39), (39, 40), (41, 44), (45, 53), (54, 62), (63, 68), (69, 77), (78, 80), (81, 82), (83, 88), (89, 93), (93, 96), (97, 99), (100, 103), (104, 113), (114, 119), (120, 123), (124, 127), (128, 133), (134, 140), (141, 146), (146, 147), (148, 149), (150, 152), (152, 153), (153, 154), (154, 155), (156, 161), (162, 168), (168, 169), (170, 172), (173, 182), (182, 183), (183, 184), (185, 189), (190, 194), (195, 197), (198, 205), (206, 208), (208, 209), (210, 214), (214, 215), (216, 217), (218, 220), (220, 221), (221, 222), (222, 223), (224, 229), (230, 236), (237, 240), (241, 249), (250, 252), (253, 261), (262, 264), (264, 265), (266, 270), (271, 273), (274, 277), (278, 284), (285, 291), (291, 292), (293, 296), (297, 302), (303, 311), (312, 322), (323, 330), (330, 331), (331, 332), (333, 338), (339, 342), (343, 3

In [22]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

beyonce Beyonce


In [23]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [24]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

18 19


In [25]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

jay z
Jay Z


In [26]:
pad_on_right = tokenizer.padding_side == "right"

In [27]:
model = AutoModelForQuestionAnswering.from_pretrained("test-squad-trained")

# Move model to GPU
model.to(device)

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
      

In [28]:
# Preprocessing the training data
def prepare_train_features(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]
    tokenized_examples = tokenizer(
        examples["question" if tokenizer.padding_side == "right" else "context"],
        examples["context" if tokenizer.padding_side == "right" else "question"],
        truncation="only_second" if tokenizer.padding_side == "right" else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(tokenized_examples["offset_mapping"]):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]

        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if tokenizer.padding_side == "right" else 0):
                token_start_index += 1
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if tokenizer.padding_side == "right" else 0):
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [29]:
features = prepare_train_features(datasets['train'][:5])

In [30]:
# Map training features to GPU
tokenized_datasets = datasets.map(
    prepare_train_features,
    batched=True,
    remove_columns=datasets["train"].column_names
)
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'start_positions', 'end_positions'])

## Fine-tuning the model

In [31]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    push_to_hub=True,
)

In [32]:
data_collator = default_data_collator

In [33]:
# Define trainer
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)


In [34]:
# trainer.train()

In [35]:
# trainer.save_model("test-squad-trained")

## Evaluation

In [36]:
# Evaluation
for batch in trainer.get_eval_dataloader():
    break

batch = {k: v.to(device) for k, v in batch.items()}
with torch.no_grad():
    output = model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [37]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([64, 384]), torch.Size([64, 384]))

In [38]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([102, 127, 257,   0,  14,  34,  55,  66, 102,  31,  37,  52,  69,  10,
          45,  65,  77,  94,   0,   0,   0,   0,  31,  55, 117, 142,   0,  64,
          45,  97, 135,  30,  42,   0,   0,  26,  28,  52,  63,   0,   0,  76,
           0,  20,  24,  78, 106, 166,   0,  18,   0, 139,  50,  52,  55,  92,
         152,   0,   0,   0,  26,  75,  75,  88], device='cuda:0'),
 tensor([105,   0, 260,   0,  15,  35,  56,  71, 106,  34,  51,  53,  70,  13,
          45,  66,  80,  96,   0,   0,   0,   0,  34,  61, 118, 142,  25,  65,
          46,  99, 138,  37,  43,   0,   0,  26,  29,  59,  64,   0,   0,   0,
           0,  24,  43,  79, 106, 166,   0,  24,   0, 141,  53,  54,  55,  94,
         158,   0,   0,   0,  29,  76,  76,  93], device='cuda:0'))

In [39]:
n_best_size = 20

In [40]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

In [41]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [42]:
# Map validation features to GPU
validation_features = datasets["test"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["test"].column_names
)
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

In [43]:
# Predictions on GPU
raw_predictions = trainer.predict(validation_features)

/home/rabeea/anaconda3/envs/tor/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py:32: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 0 which
    has less than 75% of the memory or cores of GPU 1. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


In [44]:
len(raw_predictions)

3

In [45]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [46]:
max_answer_length = 30

In [47]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["test"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 15.840794, 'text': 'the group became one'},
 {'score': 13.062016, 'text': 'Knowles, the group became one'},
 {'score': 10.543509, 'text': 'Mathew Knowles, the group became one'},
 {'score': 9.170052, 'text': 'the group became one of'},
 {'score': 8.622179,
  'text': "the group became one of the world's best-selling girl groups of"},
 {'score': 8.147081, 'text': 'one'},
 {'score': 7.791733,
  'text': "the group became one of the world's best-selling girl groups"},
 {'score': 7.053434, 'text': "the group became one of the world's"},
 {'score': 6.473832,
  'text': "the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's"},
 {'score': 6.41477,
  'text': "the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release"},
 {'score': 6.3912745, 'text': 'Knowles, the group became one of'},
 {'score': 6.371533, 'text': 'the'},
 {'score': 5.8434024,
  'text': "Knowles, the group became o

In [48]:
datasets["test"][0]["answers"]

{'text': ['in the late 1990s'], 'answer_start': [269]}

In [49]:
import collections

examples = datasets["test"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [50]:
# Postprocess QA predictions
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size=20, max_answer_length=30):
    all_start_logits, all_end_logits = raw_predictions
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)

    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    predictions = collections.OrderedDict()

    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    for example_index, example in enumerate(tqdm(examples)):
        feature_indices = features_per_example[example_index]
        min_null_score = None
        valid_answers = []
        context = example["context"]

        for feature_index in feature_indices:
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            offset_mapping = features[feature_index]["offset_mapping"]

            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]

            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            start_indexes = np.argsort(start_logits)[-1: -n_best_size - 1: -1].tolist()
            end_indexes = np.argsort(end_logits)[-1: -n_best_size - 1: -1].tolist()

            for start_index in start_indexes:
                for end_index in end_indexes:
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue

                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}

        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

# Map validation features to GPU
validation_features = datasets["test"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["test"].column_names
)
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

# Predictions on GPU
raw_predictions = trainer.predict(validation_features)

In [51]:
# Postprocess predictions
final_predictions = postprocess_qa_predictions(datasets["test"], validation_features, raw_predictions.predictions)

Post-processing 130319 example predictions split into 131754 features.


  0%|          | 0/130319 [00:00<?, ?it/s]

In [58]:
metric = load_metric("squad_v2" if squad_v2 else "squad")

/tmp/ipykernel_1518955/2905994612.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2" if squad_v2 else "squad")


In [59]:
metric = load_metric("squad_v2" if squad_v2 else "squad")

if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["test"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact': 86.66119291891435,
 'f1': 91.49233744750889,
 'total': 130319,
 'HasAns_exact': 83.94397668766773,
 'HasAns_f1': 91.19556240796064,
 'HasAns_total': 86821,
 'NoAns_exact': 92.0846935491287,
 'NoAns_f1': 92.0846935491287,
 'NoAns_total': 43498,
 'best_exact': 86.72641748325263,
 'best_exact_thresh': 0.0,
 'best_f1': 91.55756201184958,
 'best_f1_thresh': 0.0}

In [61]:
import json
# Specify the file path where you want to save the data
file_path = "references.json"

# Save data to JSON file
with open(file_path, "w") as json_file:
    json.dump(references, json_file)
# Specify the file path where you want to save the data
file_path = "formatted_predictions.json"

# Save data to JSON file
with open(file_path, "w") as json_file:
    json.dump(formatted_predictions, json_file)

In [65]:
# Function to calculate F1 score
def compute_f1_score(prediction, reference):
    predicted_tokens = tokenizer.tokenize(prediction)
    reference_tokens = tokenizer.tokenize(reference)
    common_tokens = set(predicted_tokens) & set(reference_tokens)
    all_tokens = set(predicted_tokens) | set(reference_tokens)
    precision = len(common_tokens) / len(all_tokens) if len(all_tokens) > 0 else 0.0
    recall = len(common_tokens) / len(reference_tokens) if len(reference_tokens) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0.0
    return f1

In [70]:
from sklearn.metrics import f1_score
import numpy as np

# Calculate F1 score for each example
f1_scores = []
for pred in tqdm(formatted_predictions,desc='processing'):
    reference = next(ref for ref in references if ref["id"] == pred["id"])
    reference_text = reference["answers"]["text"] if reference["answers"]["text"] else ['']
#     print((reference_text, pred["prediction_text"]))
    f1 = compute_f1_score(reference_text[0], pred["prediction_text"])
    f1_scores.append({"id": pred["id"], "f1_score": f1})

# Sort examples based on descending F1 scores
sorted_examples = sorted(f1_scores, key=lambda x: x["f1_score"], reverse=True)

processing:   0%|          | 0/130319 [00:00<?, ?it/s]

In [71]:
# Define the scoring functions
epsilon = 0.01

def hard_score(f1):
    return 1 - f1 + epsilon

def moderate_score(f1):
    return 2 - f1

def soft_score(f1):
    return 3 - f1

# Calculate probability weights for each example
total_examples = len(sorted_examples)
hard_weights = [hard_score(example["f1_score"]) for example in tqdm(sorted_examples,desc='processing')]
moderate_weights = [moderate_score(example["f1_score"]) for example in tqdm(sorted_examples,desc='processing')]
soft_weights = [soft_score(example["f1_score"]) for example in tqdm(sorted_examples,'processing')]

# Normalize weights
hard_weights /= np.sum(hard_weights)
moderate_weights /= np.sum(moderate_weights)
soft_weights /= np.sum(soft_weights)

processing:   0%|          | 0/130319 [00:00<?, ?it/s]

processing:   0%|          | 0/130319 [00:00<?, ?it/s]

processing:   0%|          | 0/130319 [00:00<?, ?it/s]

In [72]:
# Sample 5000 examples based on the computed weights
num_samples = 5000
sampled_indices = np.random.choice(total_examples, size=num_samples, p=hard_weights, replace=True)
sampled_examples = [sorted_examples[i] for i in tqdm(sampled_indices,desc='processing')]

# Extract the sampled IDs
sampled_ids = [example["id"] for example in tqdm(sampled_examples,desc='processing')]

# Retrieve the corresponding examples from the dataset
sampled_data_hard = [example for example in tqdm(datasets["test"],desc='processing') if example["id"] in sampled_ids]

file_path = "sampled_data_hard.json"

# Save data to JSON file
with open(file_path, "w") as json_file:
    json.dump(sampled_data_hard, json_file)

processing:   0%|          | 0/5000 [00:00<?, ?it/s]

processing:   0%|          | 0/5000 [00:00<?, ?it/s]

processing:   0%|          | 0/130319 [00:00<?, ?it/s]

In [90]:
sdhdict = Dataset.from_pandas(pd.DataFrame(sampled_data_hard))
save_path = "dataset_hard"

# Save the dataset to the specified path
sdhdict.save_to_disk(save_path)

Saving the dataset (0/1 shards):   0%|          | 0/4769 [00:00<?, ? examples/s]

In [73]:
# Sample 5000 examples based on the computed weights
num_samples = 5000
sampled_indices = np.random.choice(total_examples, size=num_samples, p=moderate_weights, replace=True)
sampled_examples = [sorted_examples[i] for i in tqdm(sampled_indices,desc='processing')]

# Extract the sampled IDs
sampled_ids = [example["id"] for example in tqdm(sampled_examples,desc='processing')]

# Retrieve the corresponding examples from the dataset
sampled_data_moderate = [example for example in tqdm(datasets["test"],desc='processing') if example["id"] in sampled_ids]

file_path = "sampled_data_moderate.json"

# Save data to JSON file
with open(file_path, "w") as json_file:
    json.dump(sampled_data_moderate, json_file)

processing:   0%|          | 0/5000 [00:00<?, ?it/s]

processing:   0%|          | 0/5000 [00:00<?, ?it/s]

processing:   0%|          | 0/130319 [00:00<?, ?it/s]

In [91]:
sdmdict = Dataset.from_pandas(pd.DataFrame(sampled_data_moderate))
save_path = "dataset_moderate"

# Save the dataset to the specified path
sdmdict.save_to_disk(save_path)

Saving the dataset (0/1 shards):   0%|          | 0/4907 [00:00<?, ? examples/s]

In [74]:
# Sample 5000 examples based on the computed weights
num_samples = 5000
sampled_indices = np.random.choice(total_examples, size=num_samples, p=soft_weights, replace=True)
sampled_examples = [sorted_examples[i] for i in tqdm(sampled_indices,desc='processing')]

# Extract the sampled IDs
sampled_ids = [example["id"] for example in tqdm(sampled_examples,desc='processing')]

# Retrieve the corresponding examples from the dataset
sampled_data_soft = [example for example in tqdm(datasets["test"],desc='processing') if example["id"] in sampled_ids]

file_path = "sampled_data_soft.json"

# Save data to JSON file
with open(file_path, "w") as json_file:
    json.dump(sampled_data_soft, json_file)

processing:   0%|          | 0/5000 [00:00<?, ?it/s]

processing:   0%|          | 0/5000 [00:00<?, ?it/s]

processing:   0%|          | 0/130319 [00:00<?, ?it/s]

In [92]:
sdsdict = Dataset.from_pandas(pd.DataFrame(sampled_data_soft))
save_path = "dataset_soft"

# Save the dataset to the specified path
sdsdict.save_to_disk(save_path)

Saving the dataset (0/1 shards):   0%|          | 0/4895 [00:00<?, ? examples/s]